## 1. Coleta de Dados

1.1. Criação da conexão

1.2. Consulta na tabela

1.3. Inserir dados

1.4. Excluir dados

In [8]:
import pandas as pd
import oracledb
import autenticacao


In [11]:
class OperacaoBancoDados:
    def __init__(self, nome_tabela: str, nome_conexao: str):
        self.db_config = dados_autenticar["credencial_banco"]
        self.nome_tabela = nome_tabela
        self.nome_conexao = nome_conexao
        self.conexao = None

    def conectar(self):
        """Método para conectar ao banco de dados Oracle."""
        try:
            self.conexao = oracledb.connect(
                user=self.db_config['usuario'], 
                password=self.db_config['senha'], 
                dsn=self.db_config['dsn'], 
                mode=oracledb.DEFAULT_AUTH
            )
            print("Conexão com o banco de dados Oracle estabelecida com sucesso.")
        except oracledb.DatabaseError as e:
            print(f"Erro ao conectar ao banco de dados: {e}")
            self.conexao = None
        return self.conexao

    
    def executar_consulta(self, consulta: str):
        """Método para executar uma consulta SQL e retornar os dados em um DataFrame."""
        if self.conexao:
            try:
                # Carregar dados em um DataFrame usando pandas
                df = pd.read_sql(consulta, self.conexao)
                return df
            except Exception as e:
                print(f"Erro ao executar a consulta: {e}")
                return None
        else:
            print("Conexão não estabelecida.")
            return None

    def fechar_conexao(self):
        """Método para fechar a conexão com o banco de dados."""
        if self.conexao:
            self.conexao.close()
            print("Conexão com o banco de dados fechada.")
        else:
            print("Nenhuma conexão aberta para fechar.")




In [13]:
# Criando uma instância da classe e passando o nome da tabela e nome da conexão
banco_dados = OperacaoBancoDados(
    nome_tabela='T_Usuario',
    nome_conexao='ConexaoOracle'  
)

# Conectando ao banco
conexao = banco_dados.conectar()

Conexão com o banco de dados Oracle estabelecida com sucesso.


In [19]:
query_usuario = """
    SELECT * FROM T_USUARIO
"""

# Executando a consulta e recebendo os dados em um DataFrame
df_usuario = banco_dados.executar_consulta(query_usuario)

Erro ao executar a consulta: Execution failed on sql '
    SELECT * FROM T_USUARIO
': ORA-00942: table or view does not exist
Help: https://docs.oracle.com/error-help/db/ora-00942/


/var/folders/38/6x4f55x100b7srd3m42bprm80000gn/T/ipykernel_74142/1627561300.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(consulta, self.conexao)


In [16]:
df_usuario

,Id,Nome,Sobrenome,Telefone,Email,Senha
0,1,Claudio Silva,Bispo,11958757740,claudio_cssp@hotmail.com,123456


In [18]:
query_comodo = """
    SELECT * FROM T_COMODO
"""

# Executando a consulta e recebendo os dados em um DataFrame
df_comodo = banco_dados.executar_consulta(query_comodo)
df_comodo

/var/folders/38/6x4f55x100b7srd3m42bprm80000gn/T/ipykernel_74142/1627561300.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(consulta, self.conexao)


,Id,IdUsuario,Descricao


In [20]:
import pandas as pd
import random
from datetime import datetime, timedelta
import pandas as pd

# Dados de ambientes e itens
ambientes = ['sala', 'cozinha', 'quarto1', 'quarto2', 'banheiro', 'lavanderia', 'sacada']
itens = {
    'sala': ['ar-condicionado', 'TV', 'luz'],
    'cozinha': ['geladeira', 'micro-ondas', 'luz'],
    'quarto1': ['ar-condicionado', 'luz'],
    'quarto2': ['computador', 'luz'],
    'banheiro': ['aquecedor', 'luz'],
    'lavanderia': ['máquina de lavar'],
    'sacada': ['luz']
}

##Turnos:
##Turno 1 (Manhã): das 06:00 às 12:00
##Turno 2 (Tarde): das 12:00 às 18:00
##Turno 3 (Noite): das 18:00 às 00:00
##Turno 4 (Madrugada): das 00:00 às 06:00

# Função para gerar o consumo diário
def gerar_consumo_diario():
    # Consumo entre 0.5 kWh e 5 kWh
    return round(random.uniform(0.5, 5.0), 2)

# Função para calcular o custo baseado em tarifas diferenciadas
def calcular_custo(consumo_kwh, turno):
    # Tarifas (podem ser ajustadas conforme os turnos)
    tarifas = {
        'manha': 0.60,  # Turno Manhã: R$ 0,60 por kWh
        'tarde': 0.60,  # Turno Tarde: R$ 0,60 por kWh
        'noite': 0.75,  # Turno Noite (Ponta): R$ 0,75 por kWh
        'madrugada': 0.50  # Turno Madrugada: R$ 0,50 por kWh
    }
    
    # Aplique a tarifa do turno
    tarifa = tarifas.get(turno, 0.60)  # Se o turno não for encontrado, assume tarifa de manhã/tarde
    
    # Impostos e taxas (porcentagens)
    icms = 0.18  # ICMS (18%)
    pis_cofins = 0.0384  # PIS/COFINS (3,84%)
    outros_encargos = 0.05  # Outros encargos (5%)
    
    # Calculando o custo
    custo_base = consumo_kwh * tarifa
    custo_impostos = custo_base * (icms + pis_cofins)
    custo_encargos = custo_base * outros_encargos
    
    # Cálculo final com impostos e encargos
    custo_total = custo_base + custo_impostos + custo_encargos
    return round(custo_total, 2)

# Função para atribuir o turno baseado na data (sem considerar hora exata)
def determinar_turno(data):
    # Atribuir turnos com base no dia da semana e sua posição no mês
    if data.weekday() < 5:  # Segunda a Sexta (turnos regulares)
        if data.day % 4 == 0:
            return 'noite'  # Simular que o consumo maior é à noite (pico)
        else:
            return 'manha'  # Fora ponta, simula o consumo durante o dia
    else:  # Fim de semana, o consumo pode ser aleatório
        return random.choice(['tarde', 'madrugada'])  # Simula consumo mais fora do horário de pico

# Gerar dados para 2000 linhas
dados = []
data_inicio = datetime(2023, 1, 1)
for i in range(2000):
    data_consumo = data_inicio + timedelta(days=i)
    ambiente = random.choice(ambientes)
    item = random.choice(itens[ambiente])
    consumo_diario = gerar_consumo_diario()
    
    # Determinar o turno
    turno = determinar_turno(data_consumo)
    
    # Calcular o custo com base no turno
    custo = calcular_custo(consumo_diario, turno)
    
    # Adicionar linha de dados
    dados.append({
        'data': data_consumo.strftime('%Y-%m-%d'),
        'turno': turno,
        'ambiente': ambiente,
        'item': item,
        'consumo_kwh': consumo_diario,
        'custo_estimado': custo
    })

# Criar o DataFrame
df = pd.DataFrame(dados)

# Exibir as primeiras linhas do DataFrame
df.head(10)

# Salvar os dados em um arquivo CSV
df.to_csv('historico_consumo', index=False)
